## 6.5 循环神经网络的简洁实现

In [14]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch.utils as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

### 6.5.1 定义模型

In [6]:
num_hiddens = 256
# 输入形状为(时间步数, 批量大小, 输入个数)  输入个数=one-hot向量长度(词典大小)
# rnn_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens)
rnn_layer = nn.RNN(input_size=vocab_size, hidden_size=num_hiddens)

> 1. **输入形状为(时间步数, 批量大小, 输入个数); 输入个数=one-hot向量长度(词典大小)**
> 2. **`nn.RNN`实例在前向计算后会分别返回`输出`和`隐藏状态h`**
>     - **输出指隐藏层在`各个时间步`上计算并输出的隐藏状态,通常作为后续输出层的输入**
>     - **该"输出"本身不设计输出层计算,形状为(时间步数,批量大小,隐藏单元个数)**
> 3. **`nn.RNN`实例前向计算返回的隐藏状态指:隐藏层在`最后时间步`的隐藏状态;如果有多层,每层的隐藏状态都会记录该变量中**
> 4. **对于`LSTM`,隐藏状态是一个元祖(h, c), 即hidden state和cell state**

![循环神经网络(LSTM为例)](img/6.5.png)

In [7]:
num_steps = 35
batch_size = 2
state = None

X = torch.rand(num_steps, batch_size, vocab_size)
Y, state_new = rnn_layer(X, state)
print(Y.shape, len(state_new), state_new[0].shape)
# 如果是nn.LSTM输出为: torch.Size([35, 2, 256]) 2 torch.Size([1, 2, 256])

torch.Size([35, 2, 256]) 1 torch.Size([2, 256])


In [11]:
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1)
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size, vocab_size)
        self.state = None
    
    def forward(self, inputs, state): # inputs: (batch, seq_len)
        # 获取one-hot向量表示
        X = d2l.to_onehot(inputs, self.vocab_size) # type(X) = list
        Y, self.state = self.rnn(torch.stack(X), state)
        # 全连接层会受限将Y的形状变为(num_steps * batch_size, num_hiddens), 它的输出
        # 形状为(num_steps * batch_size, vocab_size)
        output = self.dense(Y.view(-1, Y.shape[-1]))
        return output, self.state

### 6.5.2 训练模型

In [18]:
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char,
                       char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]] # output会记录prefix加上输出
    for t in range(num_chars + len(prefix) - 1):
        # X 为输入,一开始为prefix的最后一个字符
        X = torch.tensor([output[-1]], device=device).view(1, 1)
        if state is not None:
            if isinstance(state, tuple):  # LSTM, state: (h, c)
                state = (state[0].to(device), state[1].to(device))
            else:
                state = state.to(device)
        
        (Y, state) = model(X, state)
        if t < len(prefix) - 1: # 此时还是prefix的字符
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(Y.argmax(dim=1).item())
    return ''.join([idx_to_char[i] for i in output])

In [19]:
# 权重使用随机值的模型
model = RNNModel(rnn_layer, vocab_size).to(device)
predict_rnn_pytorch('分开', 10, model, vocab_size, device, idx_to_char, char_to_idx)

'分开邻意也邻滩意也邻滩邻'

In [26]:
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                 corpus_indices, idx_to_char, char_to_idx,
                                 num_epochs, num_steps, lr, clipping_theta,
                                 batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    state = None
    
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = d2l.data_iter_consecutive(corpus_indices, batch_size, num_steps, device) # 相邻采样
        for X, Y in data_iter:
            if state is not None:
                # 使用detach函数是计算图分离隐藏状态,为了
                # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                if isinstance (state, tuple): # LSTM, state: (h, c)
                    state = (state[0].detach(), state[1].detach())
                else:
                    state = state.detach()
            
            (output, state) = model(X, state) # output: 形状为(num_steps * batch_size, vocab_size)
            
            # Y的形状是(batch_size, num_steps), 转置后再变成长度为
            # batch * num_steps 的向量, 这样根输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            l = loss(output, y.long())
            
            optimizer.zero_grad()
            l.backward()
            
            # 梯度裁剪
            d2l.grad_clipping(model.parameters(), clipping_theta, device)
            optimizer.step() # 更新所有参数
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        
        try:
            perplexity = math.exp(l_sum / n)
        except OverflowError:
            perplexity = float('inf')
        
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, perplexity, time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(prefix, pred_len, model, vocab_size, device,
                                               idx_to_char, char_to_idx))
        

In [27]:
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 1.184244, time 0.42 sec
 - 分开 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活 不知不觉 你已经离开我
 - 不分开 是没有没有你在 我跟你 我不再  不懂不懂不能骑可爱女人 温柔的让我心疼的可爱女人 透明的让我感动
epoch 100, perplexity 1.041070, time 0.43 sec
 - 分开 我有了口生活 我知道你安很  从再想通 我想就这样打我妈妈 我说的话 你甘会听 不要再这样打我妈妈
 - 不分开不 我不能再爱你 会学人开过 你能再感动 想想就 让我来到 你着我妈妈  不知不觉 你怎么打我手 你
epoch 150, perplexity 1.019878, time 0.40 sec
 - 分开 我有了口不能 平知道不多难熬 是你爱  我想 难不听 不知太觉 我已经离开我 不知不觉 我跟了这节
 - 不分开不想多多 没有你有的打我妈 就能为每天在这有飞不能承受我已无处可躲 我不要再想 我不要再想 我不 我
epoch 200, perplexity 1.014650, time 0.42 sec
 - 分开 我有了口不能 平 沙不后 也能再梦 在慢管知不觉 我想了这样打我妈妈 我不知 这样没有喝水 我我 
 - 不分开不想多多 就是是不是你爱想 这样打我妈妈 我不怎么  我爱你看 打我妈 说不听 语言我 被能不到 我
epoch 250, perplexity 1.031919, time 0.41 sec
 - 分开 我以让 干什么 干什么 干什么 干什么 日行千里系沙袋 飞檐走壁莫奇怪 去去就来 一个马步向前 一
 - 不分开不想多不说 这样这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活 不知不觉 你已
